# 情感分析（第 2 部分）


## 第 5 步：切换工具 - RNN

我们刚刚了解了如何通过传统的机器学习方法解决情感分析任务：词袋 + 非线性分类器。我们现在将切换工具，并使用递归神经网络（具体而言是 LSTM）在 Keras 中执行情感分析。幸运的是，Keras 具有内置的 [IMDb 影评数据集](https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification)供我们使用，并且词汇量一样。

In [1]:
from keras.datasets import imdb  # import the built-in imdb dataset in Keras

# Set the vocabulary size
vocabulary_size = 5000

# Load in training and test data (note the difference in convention compared to scikit-learn)
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocabulary_size)
print("Loaded dataset with {} training samples, {} test samples".format(len(X_train), len(X_test)))

Using TensorFlow backend.
D:\setup_space_all\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\setup_space_all\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\setup_space_all\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\setup_space_all\anaconda\lib\site-packages\tensorflow\python\framewo

ValueError: Object arrays cannot be loaded when allow_pickle=False

In [ ]:
# Inspect a sample review and its label
print("--- Review ---")
print(X_train[7])
print("--- Label ---")
print(y_train[7])

注意标签是整数（0 表示负面，1 表示正面），影评本身存储为整数序列。它们是预先分配给每个单词的单词 ID。要将它们映射回原始单词，可以使用 `imdb.get_word_index()` 返回的字典。

In [ ]:
# Map word IDs back to words
word2id = imdb.get_word_index()
id2word = {i: word for word, i in word2id.items()}
print("--- Review (with words) ---")
print([id2word.get(i, " ") for i in X_train[7]])
print("--- Label ---")
print(y_train[7])

In [ ]:
len_BOW = [len(l) for l in X_train]
max(len_BOW), min(len_BOW)

词袋方法直接将每个单词的数量汇总到一个文档中，而此表示法本质上保留了整个单词序列（没有标点、停用词等）。这对 RNN 的正常运转来说很关键。但是也意味着 __特征可能长度不一__ 了！

#### 问题：长度不等的影评

训练集中最长的影评有多长（用单词数量计），最短的呢？

#### √答案：

(2494, 11)

### TODO：填充序列

为了将此数据馈送到 RNN 中，所有输入文档必须具有相同的长度。我们将最大影评长度限定为 `max_words`，截断更长的影评并用空值 (0) 填充更短的影评。你可以在 Keras 中使用 [`pad_sequences()`](https://keras.io/preprocessing/sequence/#pad_sequences) 函数轻松实现这一点。暂时，我们将 `max_words` 设为 500。

In [ ]:
from keras.preprocessing import sequence

# Set the maximum number of words per document (for both training and testing)
max_words = 500

# TODO: Pad sequences in X_train and X_test
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)

In [ ]:
X_train[0].shape

### TODO：设计情感分析 RNN 模型

请在下面的代码单元格中构建你的模型架构。我们从 Keras 中导入了一些你可能需要的层级，但是你也可以选择使用任何其他层级/转换。

注意，输入是单词序列（严格来说，是整数单词 ID），最大长度是 `max_words`，输出是二元情感标签（0 或 1）。

In [2]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout # Embedding: a kind of word2vec

# TODO: Design your model
# embedding_size = 32
model = Sequential()
model.add(Embedding(vocabulary_size, 32, input_length=max_words))      # count of samples, embeding output size, 
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

model.summary()

NameError: name 'max_words' is not defined

#### 问题：架构和参数

请简要描述下你的神经网络架构。它有多少模型参数需要训练？

#### √答案：

160000 params

### TODO：训练并评估模型

现在你已经准备好训练模型了。在 Keras 环境中，你首先需要 __编译__ 模型，方法是指定训练期间要使用的损失函数和优化器，以及要衡量的任何评估指标。请指定相应的参数，至少包括指标 `'accuracy'`。

In [8]:
# TODO: Compile your model, specifying a loss function, optimizer, and metrics
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

编译后，可以开始训练流程了。你需要指定两个重要的训练参数：**批次大小**和**训练周期数**，它们和模型架构一起决定了总的训练时间。

训练可能需要一段时间，喝杯咖啡，或者去徒步一会吧！如果可行的话，考虑使用 GPU，因为在 CPU 上训练一次可能需要好几个小时。

> **提示**：你可以从训练集中拿出一小部分数据，在训练中用作验证集。这样有助于监控训练流程并发现潜在的过拟合问题。你可以使用 `validation_data` 参数将验证集提供给 `model.fit()`，或者直接指定 `validation_split`，让 Keras 为此拿出一部分训练数据（通常是 5-10%）。在每个周期结束时评估验证指标一次。

In [9]:
# TODO: Specify training parameters: batch size and number of epochs
batch_size = 64
num_epochs = 3

# TODO(optional): Reserve/specify some training data for validation (not to be used for training)
X_valid, y_valid = X_train[:batch_size], y_train[:batch_size]  # first batch_size samples
X_train2, y_train2 = X_train[batch_size:], y_train[batch_size:]  # rest for training

# TODO: Train your model
model.fit(X_train2, y_train2,
          validation_data=(X_valid, y_valid),
          batch_size=batch_size, epochs=num_epochs)

Instructions for updating:
Use tf.cast instead.
Train on 24936 samples, validate on 64 samples
Epoch 1/3
24936/24936 [==============================] - 386s 15ms/step - loss: 0.4267 - acc: 0.7965 - val_loss: 0.2412 - val_acc: 0.9375
Epoch 2/3
24936/24936 [==============================] - 384s 15ms/step - loss: 0.2910 - acc: 0.8820 - val_loss: 0.3244 - val_acc: 0.8906
Epoch 3/3
24936/24936 [==============================] - 405s 16ms/step - loss: 0.2314 - acc: 0.9095 - val_loss: 0.3043 - val_acc: 0.8906


In [12]:
import os
# Save your model, so that you can quickly load it in future (and perhaps resume training)
model_file = "rnn_model.h5"  # HDF5 file
model.save(os.path.join('./cache/sentiment_analysis', model_file))

# Later you can load it using keras.models.load_model()
#from keras.models import load_model
#model = load_model(os.path.join(cache_dir, model_file))

训练模型后，看看它在未见过的测试数据上的表现如何。

In [13]:
# Evaluate your model on the test set
scores = model.evaluate(X_test, y_test, verbose=0)  # returns loss and other metrics specified in model.compile()
print("Test accuracy:", scores[1])  # scores[1] should correspond to accuracy if you passed in metrics=['accuracy']

Test accuracy: 0.86804


In [14]:
0.86804 / 0.83156 - 1

0.0438693539852808

#### 问题：比较 RNN 和传统方法

与词袋 + 梯度提升决策树相比，你的 RNN 模型表现如何？

#### 答案：
Accuracy: 0.86804(0.83156) Optimized Degree: 4.38%

## 延伸

你可以通过多种方式继续完善此 notebook。每种方式都有一些挑战性，但是会带来非常有意义的经验。

- 首先是通过尝试 __不同的架构、层级和参数__ ，提高模型的准确率。在没有花费过长训练时间的情况下，准确率能达到多少？ __如何防止过拟合？__ 

- 然后，你可能需要 __将模型部署为移动应用或网络服务__ 。如果要进行此类部署，你需要执行哪些操作？如何接受新的影评、将其转换为适合模型的格式，并进行实际预测？（注意，你在训练过程中使用的环境可能不可用了。）

- 对于此 notebook，我们做出了一个简化：将任务限制为二元分类任务。数据集实际上包含更精细的评分，由每个影评的文件名表示（格式为 `<[id]_[rating].txt>`，其中 `[id]` 是唯一标识符，`[rating]` 的范围是 1-10；注意中性评分 > 4 或 < 7 已被排除）。如何修改 notebook，以对评分执行回归算法？在什么情形下，回归比分类更有用，相反情况呢？

无论你朝着哪个方向发展，都确保与其他学员分享你的结果和学习成果，将结果分享到博客、论坛上，并参与在线竞争项目。这也是吸引潜在雇主的极佳方式！